In [7]:
!pip install librosa pretty_midi
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install miditok

Looking in indexes: https://download.pytorch.org/whl/cu118
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.5 MB/s eta 0:00:00


In [8]:
import os
import torch
import librosa
import pretty_midi
from google.colab import drive
from torch.utils.data import Dataset
import pandas as pd
import miditok

In [11]:
TOKENIZER_PARAMS = {
"pitch_range": (21, 109),
"beat_res": {(0,4):12, (4,12):12},
"special_tokens": ["PAD", "BOS", "EOS", "MASK"],
"use_chords": True,
"use_rests": False,
"use_tempos": True,
"use_time_signatures": True,
"use_programs": False,
"use_microtiming": True,
"ticks_per_quarter": 320,
"max_microtiming_shift": 0.125,
"num_microtiming_bins": 30,
}
config = miditok.TokenizerConfig(**TOKENIZER_PARAMS)

tokenizer=miditok.PerTok(config)

In [4]:
drive.mount('/content/drive')
DATA_ROOT='content/drive/MyDrive/song2cover'

Mounted at /content/drive


In [12]:
class AudioMidiDataset(Dataset):
  def __init__(self, root_dir, tokenizer, transform=None):
    self.root_dir = root_dir
    self.transform = transform
    df = pd.read_csv(os.path.join(root_dir, 'manifest.csv'))
    self.items = pd.to_dict('records')
    self.tokenizer = tokenizer

  def __len__(self):
    return len(self.items)

  def __getitem__(self, idx):
    item = self.items[idx]
    audio_path = item['audio_filepath']
    midi_path = item['midi_filepath']
    diff = torch.tensor(float(item['difficulty'])).float()

    y, sr = librosa.load(audio_path, sr=44100)
    stft = librosa.stft(y)
    x = torch.tensor(stft).unsqueeze(0).float()

    pm = pretty_midi.PrettyMIDI(midi_path)

    tokens = self.tokenizer.midi_to_tokens(pm)[0]
    y_tokens = torch.tensor(tokens, dtype=torch.long)

    sample = {
        'x': x,
        'y': y_tokens,
        'diff': diff
    }

    if self.transform:
      sample = self.transform(sample)

    return x, y_tokens, diff